# Getting Started with MongoDB Atlas and Python

### Overview

This is a companion notebook for this [TODO - Quick start guide](#)
It will demonstrate the following:

- 👉 Connect to Atlas instance
- 👉 Run a few sample queries


### What you need to run this notebook

- a (free) MongoDB Atlas Account
- An Atlas instance running in the cloud with sample data loaded
- and connection credentials

Follow this [TODO quick start guide](#) to set this up before proceeding.

### How to run

This notebook can be run on Google Colab and stand alone python development environments.  Click here to run on colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-1-atlas-setup/atlas-test.ipynb)


References:

- https://www.mongodb.com/languages/python/pymongo-tutorial

## Step-1: Setup

We need to setup Atlas connection credentials to run

### Option 1A - If running on Colab

- Click on 'Colab secrets' icon (🔑) on left pane
- Create a secret named `ATLAS_URI` and paste in your Atlas connection URL.  Refere to quick start guide above to find this value.  Make sure it the `notebook access` button is turned on.
- See screenshot below for example

![](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/images/colab-secret-1.png?raw=true)


### Option 1B - If running on local python environment

- setup your local python env following this [setup guide](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/setup-python-env.md)
- Create a file named `.env` in the same location as notebook
- Create an entry like this.  Replace the URI with your own Atlas connecct string

```text
ATLAS_URI=mongodb+srv://<username>:<password>@sandbox.....
```

In [1]:
# We will keep all global variables in an object to not pullute the global namespace.
class MyConfig(object):
    pass

MY_CONFIG = MyConfig()

## Step-2: Determine Runtime Environment

In [2]:

# are we running in Colab?
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   MY_CONFIG.RUNNING_IN_COLAB = True
else:
   print("NOT running in Colab")
   MY_CONFIG.RUNNING_IN_COLAB = False

NOT running in Colab


## Step-3: Install dependencies (if necessary)

We will install required libraries in cloud environments like Google Colab.  For local environments, we assume the dependencies are already setup.

In [3]:
if MY_CONFIG.RUNNING_IN_COLAB:
  # running on colab
  !pip install  \
                python-dotenv==1.0.0  \
                pymongo[srv]

## Step-4 - Load Credentials

In [4]:
## Load settings based on where we are running
##  - if runninning on google Colab, load from secrets
##  - if running locally use dotenv

if MY_CONFIG.RUNNING_IN_COLAB:
  from google.colab import userdata
  MY_CONFIG.ATLAS_URI = userdata.get('ATLAS_URI')
else:
  import os, sys
  from dotenv import find_dotenv, dotenv_values

  this_dir = os.path.abspath('')
  parent_dir = os.path.dirname(this_dir)
  sys.path.append (os.path.abspath (parent_dir))

  config = dotenv_values(find_dotenv())
  # debug
  # print (config)
  MY_CONFIG.ATLAS_URI = config.get('ATLAS_URI')
## --- end load config

## If you just want to quickly set the config manually, you can do so here.
# MY_CONFIG.ATLAS_URI = ''

if not MY_CONFIG.ATLAS_URI:
    raise Exception ("'ATLAS_URI' is not set.  Please set it above to continue...")

## Step-5: Atlas Client

Here is a handy class that has common Atlas functions.

For full implementation [see here](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/AtlasClient.py)

In [5]:
from pymongo import MongoClient

class AtlasClient ():

    def __init__ (self, altas_uri, dbname):
        self.mongodb_client = MongoClient(altas_uri)
        self.database = self.mongodb_client[dbname]

    ## A quick way to test if we can connect to Atlas instance
    def ping (self):
        self.mongodb_client.admin.command('ping')

    def get_collection (self, collection_name):
        collection = self.database[collection_name]
        return collection

    def find (self, collection_name, filter = {}, limit=0):
        collection = self.database[collection_name]
        items = list(collection.find(filter=filter, limit=limit))
        return items

## Step-6: Test Atlas Connectivity

See if we can connect to our Atlas cloud instance.

If this step fails, make sure 'connect from anywhere' is enabled on your Atlas network configuration

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-connect-2.png)

In [6]:
MY_CONFIG.DB_NAME = 'sample_mflix'
MY_CONFIG.COLLECTION_NAME = 'embedded_movies'

atlas_client = AtlasClient (MY_CONFIG.ATLAS_URI, MY_CONFIG.DB_NAME)
atlas_client.ping()
print ('Connected to Atlas instance! We are good to go!')

Connected to Atlas instance! We are good to go!


## Step-7: Sample Query of Movies

Now that we are connected to our instance.  Let's try some queries

In [7]:
print ('======== Finding some sample movies ========================')
movies = atlas_client.find (collection_name=MY_CONFIG.COLLECTION_NAME, limit=5)
print (f"Found {len (movies)} movies")
for idx, movie in enumerate (movies):
    print(f'{idx+1}\nid: {movie["_id"]}\ntitle: {movie["title"]},\nyear: {movie["year"]}\nplot: {movie["plot"]}\n')
print ('================================')

======== Finding some sample movies ========================
Found 5 movies
1
id: 573a1390f29313caabcd5293
title: The Perils of Pauline,
year: 1914
plot: Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...

2
id: 573a1391f29313caabcd68d0
title: From Hand to Mouth,
year: 1919
plot: A penniless young man tries to save an heiress from kidnappers and help her secure her inheritance.

3
id: 573a1391f29313caabcd820b
title: Beau Geste,
year: 1926
plot: Michael "Beau" Geste leaves England in disgrace and joins the infamous French Foreign Legion. He is reunited with his two brothers in North Africa, where they face greater danger from their...

4
id: 573a1391f29313caabcd8268
title: The Black Pirate,
year: 1926
plot: Seeking revenge, an athletic young man joins the pirate band responsible for his father's death.

5
id: 573a1391f29313caabcd8319
title: For Heave

In [8]:
print ('=======  Finding movies from year 1999 =========================')
movies_1999 = atlas_client.find(collection_name=MY_CONFIG.COLLECTION_NAME, filter={"year": 1999})
print (f"Found {len (movies_1999)} movies from year 1999.  Here is a sample...")
for idx, movie in enumerate (movies_1999[:5]):
    print(f'{idx+1}\nid: {movie["_id"]}\ntitle: {movie["title"]},\nyear: {movie["year"]}\nplot: {movie["plot"]}\n')

=======  Finding movies from year 1999 =========================
Found 81 movies from year 1999.  Here is a sample...
1
id: 573a139af29313caabcf0cfd
title: Three Kings,
year: 1999
plot: In the aftermath of the Persian Gulf War, 4 soldiers set out to steal gold that was stolen from Kuwait, but they discover people who desperately need their help.

2
id: 573a139af29313caabcf0e61
title: Beowulf,
year: 1999
plot: A sci-fi update of the famous 6th Century poem. In a beseiged land, Beowulf must battle against the hideous creature Grendel and his vengeance seeking mother.

3
id: 573a139af29313caabcf0e6c
title: The Mummy,
year: 1999
plot: An American serving in the French Foreign Legion on an archaeological dig at the ancient city of Hamunaptra accidentally awakens a Mummy.

4
id: 573a139af29313caabcf0e95
title: The 13th Warrior,
year: 1999
plot: A man, having fallen in love with the wrong woman, is sent by the sultan himself on a diplomatic mission to a distant land as an ambassador. Stopping

## Congrats!  Lab is Complete ! 👏